In [6]:
#!pip install flair
#!pip install vaderSentiment
#!pip install textblob 
#!pip install spacytextblob

In [32]:
import pandas as pd
import flair
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacytextblob
from textblob import TextBlob
from langdetect import detect, DetectorFactory

In [14]:
df = pd.read_csv('Corrected_Final_All.csv')

In [17]:
df.head(1)

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags
0,0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,@Diputado_Canelo Hagamos otro por el uno de ma...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,909431402.0,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']"


In [18]:
#let's use only english tweets
df = df[df['lang']=='en']

In [20]:
df.head()

,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags
16690,16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",2021-04-23,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',..."
16691,16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],2021-04-22,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']"
16692,16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],2021-04-22,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo']
16693,16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],2021-04-22,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']"
16694,16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],2021-04-22,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']"


In [45]:
#check out lang_langdetect column
df[df['lang_langdetect']=='en']['preprocessed_data_without_hashtags']

21123                                          ['youtube']
21126                                         ['confirmo']
21127    ['know', 'colombia', 'video', 'quality', 'drop...
21129                ['know', 'else', 'go', 'strike', 'ʖ']
21131                                                   []
                               ...                        
22155    ['look', 'like', 'share', 'amp', 'link', 'load...
22156    ['su', 'post', 'comentario', 'ha', 'sido', 're...
22157    ['su', 'post', 'comentario', 'ha', 'sido', 're...
22158    ['look', 'like', 'share', 'amp', 'link', 'load...
22159        ['cannot', 'troll', 'troll', 'broh', 'sorry']
Name: preprocessed_data_without_hashtags, Length: 1015, dtype: object

In [58]:
#confirm that lang_langdetect column has all English tweets
counter = 0
for i in df[df['lang_langdetect']=='en']['lang_langdetect']:
  try:
    tweet = " ".join(ast.literal_eval(i))
  except:
    tweet = None
  if tweet and detect(tweet) !='en':
    counter+=1
print(counter)

0


In [59]:
df = df[df['lang_langdetect']=='en']

### Textblob

In [130]:
#create sentiment detection function
def sentiment_textblob(df):
    testimonial = TextBlob(df['preprocessed_data_without_hashtags'])
    return 1 if testimonial.sentiment.polarity>0 else 0

In [131]:
#run sentiment detection function
df['sentiment_preprocessed_without_hashtags_textBlob'] = df.apply(sentiment_textblob,axis=1)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_preprocessed_without_hashtags_textBlob
21123,21123,2021-04-28 19:08:48,NaN,NaN,YouTube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-28,['youtube'],[''],[],NaN,Reddit,en,1.0,en,en,['youtube'],0
21126,21126,2021-04-29 0:06:33,NaN,NaN,Confirmo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-29,['confirmar'],[''],[],NaN,Reddit,en,13.0,NaN,en,['confirmo'],0
21127,21127,2021-04-29 0:29:12,NaN,NaN,You know you are in Colombia when the video qu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-29,"['you', 'know', 'you', 'are', 'in', 'colombia'...",[''],[],NaN,Reddit,en,33.0,en,en,"['know', 'colombia', 'video', 'quality', 'drop...",0
21129,21129,2021-04-29 1:00:13,NaN,NaN,You know what else is going to strike? ( ͡° ͜ʖ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-29,"['you', 'know', 'what', 'else', 'is', 'going',...",[''],[],NaN,Reddit,en,2.0,en,en,"['know', 'else', 'go', 'strike', 'ʖ']",0
21131,21131,2021-04-29 1:28:22,NaN,NaN,What?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-29,['what'],[''],[],NaN,Reddit,en,2.0,en,en,[],0


### Vader

In [128]:
#create sentiment detection function
def sentiment_vader(df):
    testimonial = SentimentIntensityAnalyzer().polarity_scores(df['preprocessed_data_without_hashtags'])
    return 1 if testimonial['compound']>0 else 0

In [133]:
#run sentiment detection function
df['sentiment_preprocessed_without_hashtags_vader'] = df.apply(sentiment_vader,axis=1)
df.head().iloc[:,-5:]

,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_preprocessed_without_hashtags_textBlob,sentiment_preprocessed_without_hashtags_vader
21123,en,en,['youtube'],0,0
21126,NaN,en,['confirmo'],0,0
21127,en,en,"['know', 'colombia', 'video', 'quality', 'drop...",0,0
21129,en,en,"['know', 'else', 'go', 'strike', 'ʖ']",0,0
21131,en,en,[],0,0


### Flair

In [86]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')

2021-09-25 23:30:45,523 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpu__gr953


100%|██████████| 265512723/265512723 [00:10<00:00, 24161458.46B/s]

2021-09-25 23:30:56,859 copying /tmp/tmpu__gr953 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-09-25 23:30:58,008 removing temp file /tmp/tmpu__gr953
2021-09-25 23:30:59,584 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [127]:
#create sentiment detection function
def sentiment_flair(df):
    sentence = Sentence(df['preprocessed_data_without_hashtags'])
    classifier.predict(sentence)
    return 1 if sentence.labels[0].to_dict()['value']=='POSTIVE' else 0
  



In [134]:
#run sentiment detection function
df['sentiment_preprocessed_without_hashtags_flair'] = df.apply(sentiment_flair,axis=1)
df.head().iloc[:,-5:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,lang_langdetect,preprocessed_data_without_hashtags,sentiment_preprocessed_without_hashtags_textBlob,sentiment_preprocessed_without_hashtags_vader,sentiment_preprocessed_without_hashtags_flair
21123,en,['youtube'],0,0,0
21126,en,['confirmo'],0,0,0
21127,en,"['know', 'colombia', 'video', 'quality', 'drop...",0,0,0
21129,en,"['know', 'else', 'go', 'strike', 'ʖ']",0,0,0
21131,en,[],0,0,0


### SpacyTextBlob

In [108]:
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 76 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [109]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [125]:
#create sentiment detection function
def sentiment_spacy(df):
    sentiment = nlp(df['preprocessed_data_without_hashtags'])
    return 1 if sentiment._.polarity>0 else 0
  



In [135]:
#run sentiment detection function
df['sentiment_preprocessed_without_hashtags_spacy']  = df.apply(sentiment_spacy,axis=1)
df.head().iloc[:,-5:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,preprocessed_data_without_hashtags,sentiment_preprocessed_without_hashtags_textBlob,sentiment_preprocessed_without_hashtags_vader,sentiment_preprocessed_without_hashtags_flair,sentiment_preprocessed_without_hashtags_spacy
21123,['youtube'],0,0,0,0
21126,['confirmo'],0,0,0,0
21127,"['know', 'colombia', 'video', 'quality', 'drop...",0,0,0,0
21129,"['know', 'else', 'go', 'strike', 'ʖ']",0,0,0,0
21131,[],0,0,0,0


### Let's take the highest voted sentiment for each tweet as final sentiment

In [139]:
sentiment_cols = ['sentiment_preprocessed_without_hashtags_textBlob',
       'sentiment_preprocessed_without_hashtags_vader',
       'sentiment_preprocessed_without_hashtags_flair',
       'sentiment_preprocessed_without_hashtags_spacy']

In [170]:
df['sentiment_score'] = df[sentiment_cols].mode(axis=1)[0].astype(int)
df.head().iloc[:,-5:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sentiment_preprocessed_without_hashtags_textBlob,sentiment_preprocessed_without_hashtags_vader,sentiment_preprocessed_without_hashtags_flair,sentiment_preprocessed_without_hashtags_spacy,sentiment_score
21123,0,0,0,0,0
21126,0,0,0,0,0
21127,0,0,0,0,0
21129,0,0,0,0,0
21131,0,0,0,0,0


In [172]:
#export completed file
df.to_csv('Corrected_Final_All_EnglishTweets_preprocessed_without_hashtags_ObinnaOnyema.csv', index=False)